# JUICE RPWI HF Geometry -- 2024/8/30

# Import lib

In [1]:
import csv
import datetime
import math
import matplotlib.pyplot as plt
import numpy as np

In [2]:
att_mode = 0                            # 0: read       1: write

In [ ]:
# 202408
data_dir    = '/Users/user/Dropbox-Univ/data/data-JUICE/geometry/'
data_name_U = 'JUICE-RPWI-HF-RWI-U_earth_20240819-20240823.csv'
data_name_V = 'JUICE-RPWI-HF-RWI-V_earth_20240819-20240823.csv'
data_name_W = 'JUICE-RPWI-HF-RWI-W_earth_20240819-20240823.csv'
data_name   = 'JUICE-RPWI-HF-RWI_earth_20240819-20240823.csv'

"""
data_name_U = 'JUICE-RPWI-HF-RWI-U_sun_20240819-20240823.csv'
data_name_V = 'JUICE-RPWI-HF-RWI-V_sun_20240819-20240823.csv'
data_name_W = 'JUICE-RPWI-HF-RWI-W_sun_20240819-20240823.csv'
data_name   = 'JUICE-RPWI-HF-RWI_sun_20240819-20240823.csv'
"""

### write from original

In [4]:
if att_mode == 1:
    file = data_dir + data_name_U
    with open(file, 'r') as f:
        reader = csv.reader(f);  i = 1
        list_U = [row for row in reader]
    file = data_dir + data_name_V
    with open(file, 'r') as f:
        reader = csv.reader(f);  i = 1
        list_V = [row for row in reader]
    file = data_dir + data_name_W
    with open(file, 'r') as f:
        reader = csv.reader(f);  i = 1
        list_W = [row for row in reader]

    if len(list_U)==len (list_V) and len (list_V)==len (list_W):
        n_num = len(list_U)
        j = 0;  k = 0
        for i in range(n_num):
            if len(list_U[i]) <10 and i < 17:
                j = i
            if len(list_U[i]) <10 and i > 17:
                k = i
                break
        list_U = list_U[j+1:k];  list_V = list_V[j+1:k];  list_W = list_W[j+1:k]
        list_U[0][1] = 'U Longitude (deg)';  list_U[0][2] = 'U Colatitude (deg)'; 
        list_V[0][1] = 'V Longitude (deg)';  list_V[0][2] = 'V Colatitude (deg)'; 
        list_W[0][1] = 'W Longitude (deg)';  list_W[0][2] = 'W Colatitude (deg)'; 
        n_num = len(list_U)

        file = data_dir + data_name
        with open(file, 'w') as f:
            writer = csv.writer(f)
            for i in range(n_num):
                writer.writerow([list_U[i][0][0:19],  list_U[i][1], list_U[i][2], list_V[i][1], list_V[i][2], list_W[i][1], list_W[i][2], list_U[i][3]])

### read

In [5]:
file = data_dir + data_name
with open(file, 'r') as f:
    reader = csv.reader(f);  i = 1
    list = [row for row in reader]

In [6]:
class struct:
    pass

data = struct()
n_num = len(list)

data.long   = np.zeros((3, n_num-1))
data.colon  = np.zeros((3, n_num-1))
data.colat  = np.zeros((3, n_num-1))
data.radius = np.zeros( n_num-1)
data.epoch  = []

for i in range(n_num):
    if i>0:
        time = datetime.datetime.strptime(list[i][0], '%Y-%m-%d %H:%M:%S')
        data.epoch.append( time )
        data.long[0][i-1] = list[i][1];  data.colat[0][i-1] = list[i][2]
        data.long[1][i-1] = list[i][3];  data.colat[1][i-1] = list[i][4]
        data.long[2][i-1] = list[i][5];  data.colat[2][i-1] = list[i][6]
        data.radius[i-1]  = list[i][7]

data.colon[0] = data.long[1]-data.long[0]
index = np.where(data.colon[0] >=  180);  data.colon[0][index] = data.colon[0][index]-360
index = np.where(data.colon[0] <= -180);  data.colon[0][index] = data.colon[0][index]+360
data.colon[1] = data.long[2]-data.long[1]
index = np.where(data.colon[1] >=  180);  data.colon[1][index] = data.colon[1][index]-360
index = np.where(data.colon[1] <= -180);  data.colon[1][index] = data.colon[1][index]+360
data.colon[2] = data.long[0]-data.long[2]
index = np.where(data.colon[2] >=  180);  data.colon[2][index] = data.colon[2][index]-360
index = np.where(data.colon[2] <= -180);  data.colon[2][index] = data.colon[2][index]+360

In [ ]:
np.sin(data.colat[0]* math.pi/180)

In [ ]:
fig = plt.figure(figsize=(14, 11))
ax1 = fig.add_subplot(5, 1, 1);  ax2 = fig.add_subplot(5, 1, 2);  ax3 = fig.add_subplot(5, 1, 3);  ax4 = fig.add_subplot(5, 1, 4);  ax5 = fig.add_subplot(5, 1, 5)

ax1.plot(data.epoch, data.colat[0],          '-r', linewidth=1, label='U')
ax1.plot(data.epoch, data.colat[1],          '-g', linewidth=1, label='V')
ax1.plot(data.epoch, data.colat[2],          '-b', linewidth=1, label='W')
ax2.plot(data.epoch, np.sin(data.colat[0]* math.pi/180), '-r', linewidth=1, label='U')
ax2.plot(data.epoch, np.sin(data.colat[1]* math.pi/180), '-g', linewidth=1, label='V')
ax2.plot(data.epoch, np.sin(data.colat[2]* math.pi/180), '-b', linewidth=1, label='W')
ax3.plot(data.epoch, data.colon[0],          '-r', linewidth=1, label='UV')
ax3.plot(data.epoch, data.colon[1],          '-g', linewidth=1, label='VW')
ax3.plot(data.epoch, data.colon[2],          '-b', linewidth=1, label='WU')
ax4.plot(data.epoch, data.long[0],           ':r', linewidth=1, label='U');    
ax4.plot(data.epoch, data.long[1],           ':g', linewidth=1, label='V');    
ax4.plot(data.epoch, data.long[2],           ':b', linewidth=1, label='W');    
ax5.plot(data.epoch, data.radius,            '-k')
ax1.set_ylabel('colatitude (deg)');  ax2.set_ylabel('sin(colat');  ax3.set_ylabel('D-longitude (deg)');  ax4.set_ylabel('longitude (deg)');  ax5.set_ylabel('radius (km)')
ax1.legend(loc='upper right', fontsize=8); ax2.legend(loc='upper right', fontsize=8); ax3.legend(loc='upper right', fontsize=8)

E_min = '2024-08-20 18:00:00';  t_min = datetime.datetime.strptime(E_min, "%Y-%m-%d %H:%M:%S");  
E_max = '2024-08-21 05:30:00';  t_max = datetime.datetime.strptime(E_max, "%Y-%m-%d %H:%M:%S");  xlim=[t_min, t_max]
# xlim=[data.epoch[0], data.epoch[-1]];  
ax1.set_xlim(xlim);  ax2.set_xlim(xlim);  ax3.set_xlim(xlim);  ax4.set_xlim(xlim);  ax5.set_xlim(xlim)
ylim=[   0, 180];                      ax1.set_ylim(ylim)
ylim=[-0.1, 1.1];                      ax2.set_ylim(ylim)
ylim=[-180, 180];                      ax3.set_ylim(ylim)
ylim=[-180, 180];                      ax4.set_ylim(ylim)

fig.show